# New ML Active Learning Workflow
---

# Import Modules

In [1]:
import os
import sys

sys.path.insert(0, os.path.join(
    os.environ["PROJ_irox"],
    "workflow/ml_modelling/ml_convergence_plots"))

import pickle
import random
import copy

import pandas as pd
import numpy as np
import plotly.plotly as py

from ase.visualize import view
from ase import io

# #############################################################################
# #############################################################################
from catlearn.regression import GaussianProcess
import gpflow

# #############################################################################
# #############################################################################
from methods import process_data_for_plot, get_layout

# Script Inputs

In [11]:
models_list_file = "models_list_iro2.pickle"

# Read Data

In [2]:
path_i = os.path.join(
    os.environ["PROJ_irox"],
    "workflow/ml_modelling/parsing_chris_dft_data",
    "df_dft_calcs.pickle")
with open(path_i, "rb") as fle:
    df_dft_calcs = pickle.load(fle)

# #############################################################################

path_i = os.path.join(
    os.environ["PROJ_irox"],
    "workflow/ml_modelling",
    "df_oqmd_data.pickle")
with open(path_i, "rb") as fle:
    df_oqmd_data = pickle.load(fle)

df_train = pd.concat([df_oqmd_data, df_dft_calcs], sort=False)
# Removing missing data
df_train = df_train[df_train["atoms"].notnull()]
df_train = df_train.drop(["energy", "force_max", "force_sum"], axis=1)

# #############################################################################
# #############################################################################
# #############################################################################

path_i = os.path.join(
    os.environ["PROJ_irox"],
    "chris_prototypes_structures",
    "data_structures.pickle")
with open(path_i, "rb") as fle:
    df_struct = pickle.load(fle)

# #############################################################################

path_i = os.path.join(
    os.environ["PROJ_irox"],
    "chris_prototypes_structures",
    "data_prototypes.pickle")
with open(path_i, "rb") as fle:
    df_proto = pickle.load(fle)

# #############################################################################

path_i = os.path.join(
    os.environ["PROJ_irox"],
    "data/ml_irox_data",
    "unique_ids.csv")
df_ids = pd.read_csv(path_i)

# #############################################################################

path_i = os.path.join(
    os.environ["PROJ_irox"],
    "workflow/ml_modelling/00_ml_workflow/190611_new_workflow",
    "df_features_pca.pickle",
    )
with open(path_i, "rb") as fle:
    df_features_pca = pickle.load(fle)

In [3]:
stoich_i = "AB3"

df_ids = df_ids[df_ids["stoich"] == stoich_i]
# df_ids = df_ids[df_ids["stoich"] == "AB2"]

df_train = df_train.loc[
    list(set(df_train.index.tolist()) & set(df_ids[df_ids["stoich"] == stoich_i]["unique_ids"].tolist()))
    ]

df_struct = df_struct.loc[
    list(set(df_struct.index.tolist()) & set(df_ids[df_ids["stoich"] == stoich_i]["unique_ids"].tolist()))
    ]

df_features_pca = df_features_pca.loc[df_ids[df_ids["stoich"] == stoich_i]["unique_ids"]]

In [4]:
df_proto = df_proto.loc[df_ids["unique_ids"]]

# Add 'computed' column for active learning logic

In [5]:
df_train["computed"] = [False for i in range(len(df_train))]

oqmd_id = df_train[df_train["source"] == "oqmd"].iloc[0].name
df_train.at[oqmd_id, "computed"] = True

# Gaussian Process

In [6]:
def gp_model(
        train_x,
        train_y,
        df_predict=None,
        ):
    """
    
input_dim=20
# variance=
# lengthscales=
active_dims
ARD=True

    """
    k = gpflow.kernels.RBF(
        input_dim=20,
        # variance=,
        # lengthscales=,
#         active_dims=,
        ARD=True,

#         20,
#         lengthscales=0.1,

        )

    m = gpflow.models.GPR(
        train_x.values, np.vstack(train_y),
        kern=k)

    m.likelihood.variance = np.sqrt(0.00003)
    gpflow.train.ScipyOptimizer().minimize(m)

    mean, var = m.predict_y(df_features_pca)

    d = {
        "prediction": mean.flatten(),
        "uncertainty": var.flatten(),
    #         "uncertainty_with_reg": pred["uncertainty_with_reg"],
        }

    df_i = pd.DataFrame(d, index=df_features_pca.index.to_list())

    return(df_i)

In [7]:
df_train_features = pd.concat(
    [df_train, df_features_pca], axis=1,
    join_axes=[df_train.index], keys=["0", "features"])

In [8]:
%%capture

tmp_list = []
models_list = []
for i in range(df_train_features.shape[0]):
# for i in range(3):
    tmp_list.append(str(i) + " Generation")
    tmp_list.append("########################################################")
    print("##################################################################")

    df_i = df_train_features[df_train_features["0"]["computed"] == True]

    train_x = df_i["features"]
    train_y = df_i["0"]["form_e_chris"] 

    model_i = gp_model(train_x, train_y, df_predict=df_features_pca)

    # #########################################################################
    # #########################################################################    
    for i_cnt, row_i in model_i.sort_values("prediction").iterrows():
        id_i = row_i.name
        calc_avail = id_i in df_train_features.index.tolist()
        if calc_avail:
            row_j = df_train_features.loc[id_i]
            if not row_j["0"]["computed"]:
                tmp_list.append("    " + id_i + " calc not previously computed, adding to training set")
                df_train_features.at[id_i, ("0", "computed")] = True
                break

    # #########################################################################
    # #########################################################################
    model_i_tmp = pd.concat(
        [
            model_i,
            pd.DataFrame(df_train_features["0"]["computed"]),
            ],
        axis=1,
        join_axes=[model_i.index])

    model_i_tmp = model_i_tmp.fillna(value={'computed': False})
    model_i = model_i_tmp

    # #########################################################################
    # #########################################################################
    models_list.append(model_i)

# #############################################################################
# #############################################################################
with open("models_list.pickle", "wb") as fle:
    pickle.dump(models_list, fle)

# for i in tmp_list:
#     print(i)

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 1.166624
  Number of iterations: 6
  Number of functions evaluations: 7


...

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: -1.180326
  Number of iterations: 76
  Number of functions evaluations: 89


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: -1.180326
  Number of iterations: 76
  Number of functions evaluations: 89


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: -1.365164
  Number of iterations: 64
  Number of functions evaluations: 73


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: -1.365164
  Number of iterations: 64
  Number of functions evaluations: 73


KeyboardInterrupt: 

In [9]:
# assert False

In [10]:
import pickle
import os


path_i = os.path.join(
    os.environ["PROJ_irox"],
    "/workflow/ml_modelling/00_ml_workflow/190611_new_workflow",
#     "df_dft_calcs.pickle",
    models_list_file,
    )

# path_i = os.path.join(
#     "models_list.pickle")

with open(path_i, "rb") as fle:
    models_list = pickle.load(fle)

FileNotFoundError: [Errno 2] No such file or directory: 'models_list.pickle'

In [ ]:
models_list[-2]

In [ ]:
data_final = []

# for i_cnt, model_i in enumerate(models_list):
for i_cnt, model_i in enumerate(models_list[-5:]):
    data_i = process_data_for_plot(
        model_i.sort_values("prediction", axis=0, inplace=False),
        color0='rgba(30,40,50,0.5)',
        color1='rgba(30,40,50,0.5)',
#         name='1st_gen',
        name=str(i_cnt),
        energy_key='prediction',
    #     marker_color_mode='array',
        marker_color_mode='single',
    #     marker_color_key='color_order',
        uncertainty_key='uncertainty',
        text_key='id',
        process_text_for_hover=False,
        explicit_error_bars=True,
        filled_error_traces=False,
        )
    data_final += data_i

# Plotting

In [ ]:
layout = get_layout(
    df=model_i,
    energy_key="prediction",
    uncertainty_key="uncertainty")

py.iplot(
    dict(data=data_final, layout=layout),
    filename=os.path.join(
        "__old__",
        "ml_convergence",
        "iro2",
        "iro2_ml_convergence_gen_0"))

In [ ]:
assert False

In [ ]:
# df_train_features

df_train_features.sort_values(("0", "form_e_chris"))

# df_train_features["0"]["form_e_chris"]

In [ ]:
# id_i = "brbizonjmy"

# id_i = "m29j648g6i"
id_i = "7ymj7t9l8p"

atoms_i = df_struct.loc[id_i]["atoms"]
io.write("__old__/7ymj7t9l8p.cif", atoms_i)

df_proto.loc[id_i]

In [ ]:
df_proto_i = df_proto[df_proto["spacegroup_i"] == 136]
display(df_proto_i)

display(
    df_struct.loc[df_proto_i.index]
    )

df_ids[df_ids["unique_ids"].isin(df_proto_i.index)]

In [ ]:
# model_i = models_list[0]
# # next_id = model_i.sort_values("prediction").iloc[0].name



# # calc_avail = next_id in df_train_features.index.tolist()
# # print(calc_avail)
# # if calc_avail:
# #     tmp = 42

# # next_id
# # df_train_features

# # df_train.at[next_id, "computed"] = True

# models_list[0].sort_values("prediction", axis=0, inplace=False)

# df_tmp = df_tmp.sort_values(
#     "prediction",
#     axis=0,
#     inplace=False)

# df_tmp = df_tmp.sort_values(
#     "prediction",
#     axis=0,
#     inplace=False)

# list(df_tmp)

# energy_key="prediction"
# uncertainty_key="uncertainty"
# df = df_tmp

# (df[energy_key] - df[uncertainty_key]).min()
# (df[energy_key] + df[uncertainty_key]).max()

# df_tmp["prediction"].min()
# df_tmp["prediction"].max()

# data_final = []
# data_i = process_data_for_plot(
#     df_tmp,
#     color0='rgba(30,40,50,0.5)',
#     color1='rgba(30,40,50,0.5)',
#     name='1st_gen',

#     energy_key='prediction',
# #     marker_color_mode='array',
#     marker_color_mode='single',
# #     marker_color_key='color_order',
#     uncertainty_key='uncertainty',
#     text_key='id',
#     process_text_for_hover=False,
#     explicit_error_bars=True,
#     filled_error_traces=False,
#     )
# data_final += data_i

In [ ]:
# # df_train_features = pd.concat(
# #     [df_train, df_features_pca], axis=1,
# #     join_axes=[df_train.index], keys=["0", "features"])

# model_i = models_list[0]

# model_i_tmp = pd.concat(
#     [
#         model_i,
#         pd.DataFrame(df_train_features["0"]["computed"]),
#         ],
#     axis=1,
#     join_axes=[models_list[0].index]
#     )

# values = {
#     'computed': False,
# #     'B': 1,
# #     'C': 2,
# #     'D': 3,
#     }
# model_i_tmp.fillna(value=values)

In [ ]:
# k = gpflow.kernels.RBF(1, lengthscales=0.1)
# m = gpflow.models.GPR(
#     train_x, train_y,
# #     np.vstack(std['train']),
# #     train_targets['target'],
#     kern=k)

# m.likelihood.variance = np.sqrt(0.00003)
# gpflow.train.ScipyOptimizer().minimize(m)
# mean, var = m.predict_y(std['test'])

# # # Scale predictions back to the orginal scale.
# # mean = mean * train_targets['std'] + train_targets['mean']

# # std_i = (var ** 0.5) * train_targets['std']
# # opt_upper = mean + std_i * tstd
# # opt_lower = mean - std_i * tstd

In [ ]:
# %%capture

# indices_cpy = copy.deepcopy(df_train_features.index.to_list())

# random.shuffle(indices_cpy)

# models_list = []
# for i in range(1, df_train.shape[0], 2):
#     df_i = df_train_features.loc[indices_cpy[0:i]]

#     train_x = df_i["features"]
#     train_y = df_i["0"]["form_e_chris"] 
#     print("---------------------------------------------------------------------------------------------")

#     models_list.append(
#         gp_model(train_x, train_y, df_predict=df_features_pca)
#         )

# with open("models_list.pickle", "wb") as fle:
#     pickle.dump(models_list, fle)

In [ ]:

#  ██████  █████  ████████ ██      ███████  █████  ██████  ███    ██
# ██      ██   ██    ██    ██      ██      ██   ██ ██   ██ ████   ██
# ██      ███████    ██    ██      █████   ███████ ██████  ██ ██  ██
# ██      ██   ██    ██    ██      ██      ██   ██ ██   ██ ██  ██ ██
#  ██████ ██   ██    ██    ███████ ███████ ██   ██ ██   ██ ██   ████

# def gp_model(
#         train_x,
#         train_y,
#         df_predict=None,
#         ):
#     """
#     """
#     # Define initial prediction parameters.
#     # noise = 0.0042  # Regularisation parameter.

#     # sigma_l = 2.3917  # Length scale parameter.
#     # sigma_f = 0.5120  # Scaling parameter.
#     # alpha = 0.8907  # Alpha parameter.

#     noise = 0.0042  # Regularisation parameter.
#     sigma_l = 6.3917  # Length scale parameter.
#     sigma_f = 0.5120  # Scaling parameter.
#     alpha = 0.3907  # Alpha parameter.

#     kdict = [
#         {
#             'type': 'quadratic',
#             'dimension': 'single',
#             # 'dimension': 'features',
#             'slope': sigma_l,
#             'scaling': sigma_f,
#             'degree': alpha,
#             }
#         ]

#     gp = GaussianProcess(
#         kernel_list=kdict, regularization=noise, train_fp=train_x,
#         train_target=train_y, optimize_hyperparameters=True,
#         scale_data=False)

#     print('Optimized kernel:', gp.kernel_list)

#     # Optimize hyperparameters:
#     gp.optimize_hyperparameters(global_opt=True)

#     # #########################################################################
#     # #########################################################################
#     # #########################################################################
#     # #########################################################################

#     if df_predict is not None:
#         pred = gp.predict(
#     #         test_fp=df_features_pca,
#             test_fp=df_predict,
#             uncertainty=True)

#         d = {
#             "prediction": pred["prediction"],
#             "uncertainty": pred["uncertainty"],
#             "uncertainty_with_reg": pred["uncertainty_with_reg"],
#             }

#         df_i = pd.DataFrame(d, index=df_features_pca.index.to_list())
        
#         return(df_i)

In [14]:
len(df_features_pca)

269

In [12]:
df_train

,atoms,id_old,path,stoich,form_e_chris,source,energy_pa,computed
id_unique,,,,,,,,
mp7svl6j8s,"(Atom('Ir', [3.24583, 9.20096, 2.96894], index...",64,IrO3/final_opt_new1-3_sorted/64_fixed.cif,AB3,-0.262468,chris,-6.12411,False
vwbhvwmjmw,"(Atom('O', [10.10731, 8.85549, 4.95154], index...",9,IrO3/final_opt_new1-3_sorted/9_fixed.cif,AB3,-0.286017,chris,-6.17152,False
b4nrnqvl8h,"(Atom('O', [-0.0, 1.96194, 3.72373], index=0),...",215,IrO3/final_opt_new1-3_sorted/215_fixed.cif,AB3,0.043482,chris,-5.8318,False
8pmqmq9r8f,"(Atom('Ir', [0.0, 0.00216, 3.61993], index=0),...",69,IrO3/final_opt_new1-3_sorted/69_fixed.cif,AB3,-0.235805,chris,-6.11949,False
nocg9ivex1,"(Atom('Ir', [0.17767, 0.0, 2.90292], index=0),...",156,IrO3/final_opt_new1-3_sorted/156_fixed.cif,AB3,-0.246969,chris,-6.13097,False
8jvfcyvk92,"(Atom('O', [0.00012, 1.80806, 1.16624], index=...",58,IrO3/final_opt_new1-3_sorted/58_fixed.cif,AB3,0.268269,chris,-5.61136,False
bpvynr7p9w,"(Atom('O', [0.00019, 1.90438, 1.90526], index=...",38,IrO3/final_opt_new1-3_sorted/38_fixed.cif,AB3,-0.392176,chris,-6.27277,False
vlbdnoxlnh,"(Atom('O', [-0.51406, 6.07664, 2.82477], index...",88,IrO3/final_opt_new1-3_sorted/88_fixed.cif,AB3,-0.245249,chris,-6.13135,False
vt9ynkzyna,"(Atom('O', [6.22411, 0.59707, 1.26321], index=...",198,IrO3/final_opt_new1-3_sorted/198_fixed.cif,AB3,-0.257918,chris,-6.12636,False
